# Сделать 3 вида наивно байесовских анализаторов. Нужно определить вид распределения. Multinomial; Bernuli; Gaussian

In [1]:
import scipy
import numpy as np
import math
import cmath
from scipy import stats
import pandas as pd 
import re
import math
import numpy as np
import datetime
import calendar
import sklearn
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict
from math import log

In [2]:
# gaussClf will be the class that will have the Gaussian naive bayes 

class gaussClf:
    def separate_by_classes(self, X, y):
        ''' This function separates our dataset in subdatasets by classes '''
        self.classes = np.unique(y)
        classes_index = {}
        subdatasets = {}
        cls, counts = np.unique(y, return_counts=True)
        self.class_freq = dict(zip(cls, counts))
        #print(self.class_freq)
        for class_type in self.classes:
            classes_index[class_type] = np.argwhere(y==class_type)
            subdatasets[class_type] = X[classes_index[class_type], :]
            self.class_freq[class_type] = self.class_freq[class_type]/sum(list(self.class_freq.values()))
        return subdatasets
    def fit(self, X, y):
        ''' The fitting function '''
        separated_X = self.separate_by_classes(X, y)
        self.means = {}
        self.std = {}
        for class_type in self.classes:
        # Here we calculate the mean and the standart deviation from datasets
            self.means[class_type] = np.mean(separated_X[class_type], axis=0)[0]
            self.std[class_type] = np.std(separated_X[class_type], axis=0)[0]
            
    def calculate_probability(self, x, mean, stdev):
        ''' This function calculates the class probability using gaussian distribution '''
        exponent = math.exp(-((x - mean) ** 2 / (2 * stdev ** 2)))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent
    def predict_proba(self, X):
        ''' This function predicts the probability for every class '''
        self.class_prob = {cls:math.log(self.class_freq[cls], math.e) for cls in self.classes}
        for cls in self.classes:
            temp_means = self.means.copy()
            #print("LEN ",len( temp_means[0]))
            for i in range(len(self.means[0])):
                #print(X[i])
                self.class_prob[cls]+=math.log(self.calculate_probability(X[i], self.means[cls][i], self.std[cls][i]), math.e)
        self.class_prob = {cls: math.e**self.class_prob[cls] for cls in self.class_prob}
        return self.class_prob
    def predict(self, X):

        ''' This funtion predicts the class of a sample '''
        pred = []
        for x in X:
            pred_class = None
            max_prob = 0
            for cls, prob in self.predict_proba(x).items():
                if prob>max_prob:
                    max_prob = prob
                    pred_class = cls
            pred.append(pred_class)
        return pred
    

In [3]:
test=gaussClf()
iris = datasets.load_iris()
X,y = iris['data'][:,:4], iris['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
#test.startup(X_,y)
test.fit(X_train,y_train)
#print(test.predict(X_test))
predict_hand=test.predict(X_test)
#print(y_train,'\n')
print(y_test)
print(predict_hand)
print("Number of mislabeled points out of a total  points : %d" % ( (y_test != predict_hand).sum()))
#count=0
#print(" gdsdhw")
#print(len(predict_hand))


[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0
 1]
[2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 0, 1]
Number of mislabeled points out of a total  points : 4


In [4]:
gnb = GaussianNB()
clf = gnb.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_test)
print(y_pred)
print("Number of mislabeled points out of a total points : %d"% ((y_test != y_pred).sum()))

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2 2 2 1 2 1 1 2 2 2 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0
 1]
[2 1 0 2 0 2 0 1 1 1 1 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 1 1 1 2 0 2 0 0 1 2 2 1 2 1 2 1 1 2 1 1 2 1 2 1 0 2 1 1 1 1 2 0 0 2 1 0 0
 1]
Number of mislabeled points out of a total points : 4


In [5]:
y_pred = gnb.fit(X_train, y_train).predict_proba(X_test)
print(y_pred)


[[5.02810309e-298 9.91792245e-006 9.99990082e-001]
 [1.32355974e-104 9.99999998e-001 2.21243985e-009]
 [1.00000000e+000 1.97379737e-018 2.22675274e-033]
 [0.00000000e+000 1.99091194e-006 9.99998009e-001]
 [1.00000000e+000 1.03560097e-015 6.00919030e-033]
 [0.00000000e+000 2.61567517e-010 1.00000000e+000]
 [1.00000000e+000 1.09216234e-015 3.04640925e-032]
 [1.30382317e-186 9.39748706e-001 6.02512944e-002]
 [1.37314714e-190 9.92477617e-001 7.52238287e-003]
 [1.61806205e-118 9.99996830e-001 3.17015005e-006]
 [6.23363697e-266 9.56045299e-001 4.39547007e-002]
 [5.93184767e-168 9.85490107e-001 1.45098927e-002]
 [3.23724087e-167 9.99979701e-001 2.02985944e-005]
 [9.06998935e-178 9.94375343e-001 5.62465723e-003]
 [6.58566389e-178 9.99094377e-001 9.05623500e-004]
 [1.00000000e+000 1.00495398e-017 2.39340136e-035]
 [6.62084819e-167 9.98615589e-001 1.38441096e-003]
 [8.44864592e-141 9.99999390e-001 6.09783320e-007]
 [1.00000000e+000 3.67025633e-014 9.78434268e-032]
 [1.00000000e+000 5.99377394e-0

Bernuli:
P(Xi|y)=P(i|y)Xi+(1-P(i|y))(1-Xi)